In [1]:
### Vacation.Py HW6

# setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [2]:
# api keys
from api_keys import g_key

In [3]:
# load weather.py csv
data = pd.read_csv("city_weather_data.csv")

data.head()

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind,Country,Date
0,New Norfolk,-42.7826,147.0587,53.80,54.91,50,100,4.97,AU,1635713599
1,Guerrero Negro,27.9769,-114.0611,72.52,72.52,57,100,12.50,MX,1635713624
2,Vaini,-21.2000,-175.2000,78.96,78.96,83,20,10.36,TO,1635713625
3,Biak,-0.9131,122.8766,75.94,75.94,81,84,0.65,ID,1635713625
4,Hobart,-42.8794,147.3294,54.54,55.26,67,75,5.01,AU,1635713418


In [4]:
### Humidity Heatmap

humidity = data["Humidity"]
locations = data[["Latitude","Longitude"]]

gmaps.configure(api_key = g_key)
maps = gmaps.figure(center=(0,0), zoom_level=1, layout={'width': '500px','height': '500px'})
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 300, point_radius = 5, dissipating =False)
maps.add_layer(heat_layer)
maps

Figure(layout=FigureLayout(height='500px', width='500px'))

In [5]:
# weather variables as dictated by homework
city_param=data.loc[(data["Temperature"]<82) & (data["Temperature"]>72) & (data["Wind"]<5) & (data["Cloudiness"]<5)]

# null value rows
city_param=city_param.dropna(how='any')
# index reset
city_param.reset_index(inplace=True)
# data frame
city_param

,index,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind,Country,Date
0,51,Hilo,19.7297,-155.0900,79.79,83.25,69,1,3.44,US,1635713486
1,141,Kapaa,22.0752,-159.3190,79.45,82.18,68,3,1.01,US,1635713674
2,151,Shenandoah,30.4013,-91.0009,76.23,77.95,51,0,1.99,US,1635713677
3,164,Uthal,25.8072,66.6219,75.72,75.72,44,0,1.52,PK,1635713682
4,165,Saint George,37.1041,-113.5841,76.28,81.07,30,1,3.00,US,1635713437
5,447,Kibala,9.1104,18.3463,78.75,78.75,83,0,0.00,TD,1635713785
6,482,Osorno,-40.5667,-73.1500,75.51,75.51,35,0,2.30,CL,1635713501


In [6]:
### Hotels in Heatmap

# data framework
hotel = city_param.loc[:,["City","Country","Latitude","Longitude"]]
# hotel column
hotel["Hotel Name"] = ""
# data frame
hotel.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Hilo,US,19.7297,-155.0900,
1,Kapaa,US,22.0752,-159.3190,
2,Shenandoah,US,30.4013,-91.0009,
3,Uthal,PK,25.8072,66.6219,
4,Saint George,US,37.1041,-113.5841,


In [7]:
# url for api
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
# create forloop for potential favorable hotels
for index, x in hotel.iterrows():
    latitude = x["Latitude"]
    longitude = x["Longitude"]
    c_name = x["City"]
    
    params["location"] = f"{latitude},{longitude}"

    print(f"Index Number {index}: {c_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
   
    try:
        if len(results) > 0 : 
            
            if "name" in  results[0]:
                print(f"Hotel in {c_name} is {results[0]['name']}.")
                hotel_name = results[0]['name']
                hotel.loc[index, "Hotel Name"] = hotel_name
            else : 
                hotel.loc[index, "Hotel Name"] = "N/A"
        else :
            hotel.loc[index, "Hotel Name"] = "N/A"
                
    except (KeyError, IndexError):
        print("None")
       
print("End")

Index Number 0: Hilo.
Hotel in Hilo is Grand Naniloa Hotel Hilo - a DoubleTree by Hilton.
Index Number 1: Kapaa.
Hotel in Kapaa is Kauai Shores Hotel.
Index Number 2: Shenandoah.
Hotel in Shenandoah is Hampton Inn & Suites Baton Rouge - I-10 East.
Index Number 3: Uthal.
Hotel in Uthal is Al Madina Hotal.
Index Number 4: Saint George.
Hotel in Saint George is Red Lion Hotel & Conference Center St. George.
Index Number 5: Kibala.
Hotel in Kibala is hotel safari.
Index Number 6: Osorno.
Hotel in Osorno is Conrado Hotel Osorno.
End


In [9]:
# Cell "in [7]" I am still unseccesful in generating a heatmap. Will follow up with classmates and tutor om 11/2/21
# failure affecting final heatmap with hotels listed above

Figure(layout=FigureLayout(height='500px', width='500px'))